#### This is a noteboook to test image similarity compare


In [23]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model, Sequeial, applications

os.environ['CPP_TF_MIN_LOG_LEVEL'] = '2'

print(tf.__version__)

2.7.0


In [33]:
# loading pre-trained ImageNet model with out top layer

# loading inception V3
inceptionV3 = applications.inception_v3.InceptionV3(
    include_top=False,
    weights='imagenet',
    input_tensor=None,
    input_shape=(None, None, 3),
    pooling='avg'
)

inceptionV3.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_10 (InputLayer)          [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d_282 (Conv2D)            (None, None, None,   864         ['input_10[0][0]']               
                                32)                                                               
                                                                                                  
 batch_normalization_282 (Batch  (None, None, None,   96         ['conv2d_282[0][0]']             
 Normalization)                 32)                                                    

In [11]:
# loading mobilenet V3
mobileNetV3 = applications.MobileNetV3Small(
    input_shape=None,
    include_top=False,
    weights='imagenet',
    pooling='avg',
    include_preprocessing=False
)

mobileNetV3.summary()

6709248/6698480 [==============================] - 5s 1us/step
Model: "MobilenetV3small"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 Conv (Conv2D)                  (None, None, None,   432         ['input_5[0][0]']                
                                16)                                                               
                                                                                                  
 Conv/BatchNorm (BatchNormaliza  (None, None, None,   64         ['Conv[0][0]']                   
 tion)              

In [13]:
# loading DenseNet121
denseNet121 = applications.densenet.DenseNet121(
    include_top=False,
    weights='imagenet',
    input_shape=None,
    pooling='avg'
)

denseNet121.summary()

Model: "densenet121"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, None, None,   0          ['input_7[0][0]']                
 D)                             3)                                                                
                                                                                                  
 conv1/conv (Conv2D)            (None, None, None,   9408        ['zero_padding2d_2[0][0]']       
                                64)                                                     

In [28]:
def concat_and_display_2_images(image_path_1, image_path_2, shape=(224, 224), interpolation=cv2.INTER_AREA):
    """ concat 2 image horizontally and display it in the window
    """
    # load image
    img_1 = cv2.imread(image_path_1)
    img_2 = cv2.imread(image_path_2)
    # resize image
    img_1 = cv2.resize(img_1, shape, interpolation=interpolation)
    img_2 = cv2.resize(img_2, shape, interpolation=interpolation)
    # concat 2 images 
    concat_img = np.concatenate((img_1, img_2), axis=1)
    # display 2 images
    cv2.imshow('concat image', concat_img)
    cv2.waitKey(0)
    #closing all open windows 
    cv2.destroyAllWindows()

In [29]:
# display 2 cats image
cat_1, cat_2 = 'cat.1.jpg', 'cat.2.jpg'
concat_and_display_2_images(cat_1, cat_2)

In [30]:
# display 2 dogs image
dog_1, dog_2 = 'dog.30.jpg', 'dog.32.jpg'
concat_and_display_2_images(dog_1, dog_2)

In [36]:
i = layers.Input([None, None, 3], dtype = tf.uint8)
x = tf.cast(i, tf.float32)
x = applications.mobilenet.preprocess_input(x)
x = inceptionV3(x)
model = Model(inputs=[i], outputs=[x])
image = tf.image.decode_png(tf.io.read_file(cat_1))
image = tf.expand_dims(image, axis=0)
result = model(image)

In [37]:
result

<tf.Tensor: shape=(1, 2048), dtype=float32, numpy=
array([[0.05594226, 0.07179575, 0.08608073, ..., 0.2312037 , 0.98325604,
        0.10751756]], dtype=float32)>

In [39]:
image2 = tf.image.decode_png(tf.io.read_file(cat_2))
image2 = tf.expand_dims(image2, axis=0)
result2 = model(image2)

In [40]:
result2

<tf.Tensor: shape=(1, 2048), dtype=float32, numpy=
array([[0.17047125, 0.10460877, 0.17450266, ..., 0.30398032, 0.7217331 ,
        0.45822632]], dtype=float32)>

In [41]:
def consine_similarity(v1, v2):
    return tf.reduce_sum(tf.multiply(tf.nn.l2_normalize(v1), tf.nn.l2_normalize(v2)))

In [42]:
consine_similarity(result, result2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.79126686>

In [44]:
v_dog1 = tf.image.decode_png(tf.io.read_file(dog_1))
v_dog1 = tf.expand_dims(v_dog1, axis=0)
r_dog1 = model(v_dog1)

v_dog2 = tf.image.decode_png(tf.io.read_file(dog_2))
v_dog2 = tf.expand_dims(v_dog2, axis=0)
r_dog2 = model(v_dog2)

r_dog1, r_dog2

(<tf.Tensor: shape=(1, 2048), dtype=float32, numpy=
 array([[0.35208276, 0.16735186, 0.14275882, ..., 0.12377867, 0.593     ,
         0.40936527]], dtype=float32)>,
 <tf.Tensor: shape=(1, 2048), dtype=float32, numpy=
 array([[0.11207836, 0.181078  , 0.10087012, ..., 0.11267054, 0.43557113,
         0.11487758]], dtype=float32)>)

In [45]:
consine_similarity(r_dog1, r_dog2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6215941>

In [46]:
consine_similarity(result, r_dog2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.5154902>

In [47]:
consine_similarity(result2, r_dog2)

<tf.Tensor: shape=(), dtype=float32, numpy=0.6052578>